# Cloud Workshop Azure Databricks
## 07. Classification Evaluation
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.4 29/06/2020

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## Evaluating a Classification Model

In this exercise, you will create a pipeline for a classification model, and then apply commonly used metrics to evaluate the resulting classifier.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [4]:
import datetime
now = datetime.datetime.now()
print(now)

2020-07-06 08:18:37.162028

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(data)

# Split the data
splits = data.randomSplit([0.7, 0.3])

train = splits[0]
test = splits[1]

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
21,2,WN,10721,13342,26,57,1
13,1,AA,15016,12892,51,27,1
5,5,FL,10397,11433,9,4,0
22,1,US,11278,14100,35,71,1
23,4,WN,12451,10693,9,5,0
5,7,AA,11298,15016,39,42,1
4,6,UA,13930,14307,71,58,1
10,3,9E,14307,11433,68,140,1
29,7,UA,14057,14771,130,125,1
14,7,UA,14771,11292,20,42,1


### Define the Pipeline and Train the Model
Now define a pipeline that creates a feature vector and trains a classification model

In [7]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")
featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

In [8]:
lr = LogisticRegression(labelCol="Late", featuresCol="features")

In [9]:
pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])
model = pipeline.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [11]:
prediction = model.transform(test)
predicted = prediction.select("features", col("prediction").cast("Int"), col("Late").alias("trueLabel"))

predicted.show(100, truncate=False)

+---------------------------------------------+----------+---------+
features |prediction|trueLabel|
+---------------------------------------------+----------+---------+
[25.0,2.0,10.0,11.0,12.0,0.38308457711442784]|1 |1 |
[25.0,2.0,10.0,47.0,16.0,0.2537313432835821] |0 |0 |
[25.0,2.0,10.0,38.0,5.0,0.5323383084577115] |1 |1 |
[25.0,2.0,10.0,38.0,1.0,0.5572139303482587] |1 |1 |
[25.0,2.0,10.0,14.0,33.0,0.24378109452736318]|0 |0 |
[25.0,2.0,10.0,48.0,31.0,0.1990049751243781] |0 |0 |
[25.0,2.0,10.0,16.0,15.0,0.23880597014925373]|0 |0 |
[25.0,2.0,2.0,34.0,5.0,0.34328358208955223] |1 |0 |
[25.0,2.0,2.0,11.0,24.0,0.21890547263681592] |0 |0 |
[25.0,2.0,2.0,35.0,5.0,0.2537313432835821] |0 |0 |
[25.0,2.0,2.0,23.0,24.0,0.25870646766169153] |0 |1 |
[25.0,2.0,2.0,2.0,0.0,0.472636815920398] |1 |1 |
[25.0,2.0,2.0,2.0,33.0,0.6467661691542288] |1 |1 |
[25.0,2.0,2.0,2.0,27.0,0.25870646766169153] |0 |0 |
[25.0,2.0,2.0,2.0,27.0,0.25870646766169153] |0 |0 |
[25.0,2.0,2.0,2.0,9.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,2.0,22.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,2.0,22.0,0.472636815920398] |1 |1 |
[25.0,2.0,2.0,2.0,26.0,0.3880597014925373] |1 |0 |
[25.0,2.0,2.0,2.0,11.0,0.29850746268656714] |0 |0 |
[25.0,2.0,2.0,2.0,2.0,0.263681592039801] |0 |0 |
[25.0,2.0,2.0,2.0,13.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,2.0,51.0,0.47761194029850745] |1 |1 |
[25.0,2.0,2.0,2.0,1.0,0.39303482587064675] |1 |0 |
[25.0,2.0,2.0,2.0,21.0,0.47761194029850745] |1 |1 |
[25.0,2.0,2.0,2.0,38.0,0.23880597014925373] |0 |0 |
[25.0,2.0,2.0,2.0,38.0,0.3681592039800995] |1 |0 |
[25.0,2.0,2.0,2.0,40.0,0.2885572139303483] |0 |0 |
[25.0,2.0,2.0,2.0,28.0,0.5024875621890548] |1 |1 |
[25.0,2.0,2.0,52.0,5.0,0.23383084577114427] |0 |0 |
[25.0,2.0,2.0,52.0,5.0,0.6567164179104478] |1 |1 |
[25.0,2.0,2.0,12.0,24.0,0.2537313432835821] |0 |0 |
[25.0,2.0,2.0,3.0,5.0,0.5074626865671642] |1 |1 |
[25.0,2.0,2.0,3.0,5.0,0.5074626865671642] |1 |1 |
[25.0,2.0,2.0,3.0,1.0,0.23880597014925373] |0 |1 |
[25.0,2.0,2.0,3.0,31.0,0.29850746268656714] |0 |0 |
[25.0,2.0,2.0,3.0,3.0,0.582089552238806] |1 |1 |
[25.0,2.0,2.0,13.0,24.0,0.31840796019900497] |0 |1 |
[25.0,2.0,2.0,21.0,10.0,0.9402985074626865] |1 |1 |
[25.0,2.0,2.0,21.0,2.0,0.29850746268656714] |0 |1 |
[25.0,2.0,2.0,21.0,13.0,0.582089552238806] |1 |1 |
[25.0,2.0,2.0,21.0,14.0,0.4925373134328358] |1 |1 |
[25.0,2.0,2.0,21.0,6.0,0.2537313432835821] |0 |0 |
[25.0,2.0,2.0,21.0,28.0,0.43781094527363185] |1 |1 |
[25.0,2.0,2.0,66.0,2.0,0.2935323383084577] |0 |1 |
[25.0,2.0,2.0,54.0,5.0,0.21393034825870647] |0 |0 |
[25.0,2.0,2.0,0.0,10.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,0.0,5.0,0.3233830845771144] |0 |0 |
[25.0,2.0,2.0,0.0,6.0,0.527363184079602] |1 |1 |
[25.0,2.0,2.0,0.0,54.0,0.5223880597014925] |1 |1 |
[25.0,2.0,2.0,0.0,28.0,0.6567164179104478] |1 |1 |
[25.0,2.0,2.0,62.0,1.0,0.8109452736318408] |1 |1 |
[25.0,2.0,2.0,5.0,2.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,40.0,5.0,0.2885572139303483] |0 |0 |
[25.0,2.0,2.0,28.0,1.0,0.2537313432835821] |0 |0 |
[25.0,2.0,12.0,69.0,17.0,0.26865671641791045]|0 |0 |
[25.0,2.0,12.0,8.0,34.0,0.5522388059701493] |1 |1 |
[25.0,2.0,12.0,19.0,69.0,0.23383084577114427]|0 |0 |
[25.0,2.0,12.0,19.0,69.0,0.2835820895522388] |0 |0 |
[25.0,2.0,12.0,19.0,40.0,0.24875621890547264]|0 |0 |
[25.0,2.0,12.0,5.0,17.0,0.2537313432835821] |0 |0 |
[25.0,2.0,12.0,40.0,17.0,0.22388059701492538]|0 |0 |
[25.0,2.0,12.0,65.0,17.0,0.25870646766169153]|0 |0 |
[25.0,2.0,7.0,11.0,26.0,0.38308457711442784] |1 |1 |
[25.0,2.0,7.0,11.0,25.0,0.21393034825870647] |0 |0 |
[25.0,2.0,7.0,11.0,63.0,0.6069651741293532] |1 |1 |
[25.0,2.0,7.0,11.0,21.0,0.4626865671641791] |1 |1 |
[25.0,2.0,7.0,7.0,10.0,0.26865671641791045] |0 |0 |
[25.0,2.0,7.0,23.0,10.0,0.4427860696517413] |1 |1 |
[25.0,2.0,7.0,23.0,10.0,0.8208955223880597] |1 |1 |
[25.0,2.0,7.0,23.0,26.0,0.40298507462686567] |1 |1 |
[25.0,2.0,7.0,23.0,28.0,0.263681592039801] |0 |1 |
[25.0,2.0,7.0,10.0,26.0,0.2537313432835821] |0 |0 |
[25.0,2.0,7.0,10.0,14.0,0.7114427860696517] |1 |1 |
[25.

### Compute Confusion Matrix Metrics
Classifiers are typically evaluated by creating a *confusion matrix*, which indicates the number of:
- True Positives
- True Negatives
- False Positives
- False Negatives

From these core measures, other evaluation metrics such as *precision* and *recall* can be calculated.

In [13]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())

metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])


In [14]:
display(metrics)

metric,value
TP,89227.0
FP,7256.0
TN,81339.0
FN,13134.0
Precision,0.9247950416135485
Recall,0.8716894129600141


### View the Raw Prediction and Probability
The prediction is based on a raw prediction score that describes a labelled point in a logistic function. This raw prediction is then converted to a predicted label of 0 or 1 based on a probability vector that indicates the confidence for each possible label value (in this case, 0 and 1). The value with the highest confidence is selected as the prediction.

In [16]:
prediction.select("rawPrediction", "probability", "prediction", col("Late").alias("trueLabel")).show(100, truncate=False)

+--------------------------------------------+------------------------------------------+----------+---------+
rawPrediction |probability |prediction|trueLabel|
+--------------------------------------------+------------------------------------------+----------+---------+
[-1.552283301743234,1.552283301743234] |[0.17475673307466316,0.8252432669253368] |1.0 |1 |
[2.0071315391955595,-2.0071315391955595] |[0.8815438127978168,0.11845618720218316] |0.0 |0 |
[-5.290440923347157,5.290440923347157] |[0.005014268126541151,0.9949857318734587] |1.0 |1 |
[-5.968628699694992,5.968628699694992] |[0.0025512210770458157,0.9974487789229541]|1.0 |1 |
[2.248304779465477,-2.248304779465477] |[0.904504208521327,0.0954957914786731] |0.0 |0 |
[3.569052373141335,-3.569052373141335] |[0.9725899378120894,0.027410062187910703] |0.0 |0 |
[2.20085862161811,-2.20085862161811] |[0.9003265888905554,0.0996734111094445] |0.0 |0 |
[-0.052075547225109986,0.052075547225109986]|[0.4869840545154514,0.5130159454845487] |1.0 |0 |
[3.196451078469586,-3.196451078469586] |[0.9607005063710948,0.039299493628905224] |0.0 |0 |
[2.246111929998434,-2.246111929998434] |[0.9043146301153246,0.09568536988467542] |0.0 |0 |
[2.246488757524867,-2.246488757524867] |[0.9043472319130608,0.09565276808693922] |0.0 |1 |
[-3.598872255363956,3.598872255363956] |[0.02662620601985146,0.9733737939801484] |1.0 |1 |
[-7.714877424742431,7.714877424742431] |[4.459411945605714E-4,0.9995540588054395] |1.0 |1 |
[2.1569575628435382,-2.1569575628435382] |[0.8963171473139578,0.10368285268604215] |0.0 |0 |
[2.1569575628435382,-2.1569575628435382] |[0.8963171473139578,0.10368285268604215] |0.0 |0 |
[2.2253857619875115,-2.2253857619875115] |[0.9025061109761294,0.09749388902387063] |0.0 |0 |
[2.3599270807700163,-2.3599270807700163] |[0.9137200570379169,0.08627994296208316] |0.0 |0 |
[-3.3711869466551025,3.3711869466551025] |[0.03320818032490047,0.9667918196750996] |1.0 |1 |
[-1.1647020963272343,1.1647020963272343] |[0.2378139399191798,0.7621860600808202] |1.0 |0 |
[0.9725035314322295,-0.9725035314322295] |[0.7256182233693601,0.2743817766306399] |0.0 |0 |
[1.7708661679942317,-1.7708661679942317] |[0.854565354438748,0.14543464556125194] |0.0 |0 |
[2.266783090843667,-2.266783090843667] |[0.906088411238735,0.09391158876126497] |0.0 |0 |
[-3.1984144019463123,3.1984144019463123] |[0.03922543527565008,0.96077456472435] |1.0 |1 |
[-1.5507934911765417,1.5507934911765417] |[0.17497169293222453,0.8250283070677754] |1.0 |0 |
[-3.508894368367475,3.508894368367475] |[0.029060215790379332,0.9709397842096206] |1.0 |1 |
[2.7802325751913086,-2.7802325751913086] |[0.9415982353132405,0.05840176468675945] |0.0 |0 |
[-0.5310777517654266,0.5310777517654266] |[0.37026555464207994,0.62973444535792] |1.0 |0 |
[1.5273503446360257,-1.5273503446360257] |[0.8216183071912398,0.17838169280876012] |0.0 |0 |
[-4.073239490360885,4.073239490360885] |[0.016737251870871557,0.9832627481291285] |1.0 |1 |
[2.8531560143012342,-2.8531560143012342] |[0.9454815920806574,0.054518407919342704] |0.0 |0 |
[-7.972281593057321,7.972281593057321] |[3.447722772610904E-4,0.999655227722739] |1.0 |1 |
[2.3106863182347306,-2.3106863182347306] |[0.909758216639433,0.09024178336056705] |0.0 |0 |
[-4.432890354528617,4.432890354528617] |[0.011740622497151942,0.9882593775028481] |1.0 |1 |
[-4.432890354528617,4.432890354528617] |[0.011740622497151942,0.9882593775028481] |1.0 |1 |
[2.403049149525371,-2.403049149525371] |[0.917059521079043,0.082940478920957] |0.0 |1 |
[1.1852320419665032,-1.1852320419665032] |[0.765887234434525,0.23411276556547503] |0.0 |0 |
[-6.363960361431733,6.363960361431733] |[0.0017195693267834367,0.9982804306732167]|1.0 |1 |
[0.6607730210098612,-0.6607730210098612] |[0.6594340155142653,0.34056598448573466] |0.0 |1 |
[-15.357943887250695,15.357943887250695] |[2.1386003004886182E-7,0.9999997861399699]|1.0 |1 |
[0.9884550003223307,-0.9884550003223307] |[0.7287826479963362,0.2712173520036638] |0.0 |1 |
[-6.157113446728392,6.157113446728392] |[0.00211388

Note that the results include rows where the probability for 0 (the first value in the **probability** vector) is only slightly higher than the probability for 1 (the second value in the **probability** vector). The default *discrimination threshold* (the boundary that decides whether a probability is predicted as a 1 or a 0) is set to 0.5; so the prediction with the highest probability is always used, no matter how close to the threshold.

### Review the Area Under ROC
Another way to assess the performance of a classification model is to measure the area under a *received operator characteristic (ROC) curve* for the model. The **spark.ml** library includes a **BinaryClassificationEvaluator** class that you can use to compute this. A ROC curve plots the True Positive and False Positive rates for varying threshold values (the probability value over which a class label is predicted). The area under this curve gives an overall indication of the models accuracy as a value between 0 and 1. A value under 0.5 means that a binary classification model (which predicts one of two possible labels) is no better at predicting the right class than a random 50/50 guess.

In [19]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(prediction)

print ("AUC = ", auc)

AUC = 0.9481913860362188

### Review the AreaUnderPR

In [21]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderPR")
areaUnderPR = evaluator.evaluate(prediction)
print ("Area Under PR = ", areaUnderPR)

Area Under PR = 0.964403314187144